## **Objetive**

## **Loading and Settings**

In [2]:
# Imports & Configuration
from pathlib import Path
import pandas as pd
import duckdb as db

con = db.connect()
DATA_DIR = Path('data/raw')
OUTPUT_FILE = Path('data/processed/stack.csv')

print(f'[CONFIG] DATA_DIR={DATA_DIR} | OUTPUT_FILE={OUTPUT_FILE}')

[CONFIG] DATA_DIR=data\raw | OUTPUT_FILE=data\processed\stack.csv


In [3]:
# Filesystem preparation

def ensure_output_dir(path: Path) -> None:
    path.parent.mkdir(parents=True, exist_ok=True)


def list_csv_files(data_dir: Path) -> list[Path]:
    files = sorted(data_dir.glob('*.csv'))
    assert files, 'No CSV files found'
    return files


ensure_output_dir(OUTPUT_FILE)
files = list_csv_files(DATA_DIR)


for file in files:
    print(file)         

print(f'Total: {len(files)} CSV files found')

data\raw\2020 and before 2004970 rows.csv
data\raw\2021-2020 2018176 rows.csv
data\raw\2022-2021 1821488 rows.csv
data\raw\2023-2022 1772092 rows.csv
data\raw\2024-2023 1818726 rows.csv
data\raw\2025-2024 1884950 rows.csv
data\raw\2026-2025 1980801 rows.csv
Total: 7 CSV files found


In [4]:
# Scan each file schema
def scan_file(con: db.DuckDBPyConnection, file: Path):
    df = con.execute(f"""
        SELECT *
        FROM read_csv_auto('{file}', sample_size=-1)
    """).df()
    return set(df.columns), len(df)

schemas = []
row_counts = {}

for i, file in enumerate(files, start=1):
    cols, n_rows = scan_file(con, file)
    schemas.append(cols)
    row_counts[file.name] = n_rows

    print(
        f'[SCAN {i}/{len(files)}]\n'
        f'{file.name} | rows={n_rows} | cols={len(cols)}'
    )

print(f'[SCAN] Completed. Files scanned: {len(schemas)}')

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

[SCAN 1/7]
2020 and before 2004970 rows.csv | rows=2004970 | cols=39


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

[SCAN 2/7]
2021-2020 2018176 rows.csv | rows=2018176 | cols=39


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

...

[SCAN 5/7]
2024-2023 1818726 rows.csv | rows=1818726 | cols=39


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

[SCAN 6/7]
2025-2024 1884950 rows.csv | rows=1884950 | cols=39


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

[SCAN 7/7]
2026-2025 1980801 rows.csv | rows=1980801 | cols=39
[SCAN] Completed. Files scanned: 7


In [5]:
# Consolidate CSV files into a single stack.csv using common columns

def consolidate_csvs(files: list[Path], schemas: list[set], con, output: Path, row_counts: dict):

    # determine common columns
    common_fields = sorted(set.intersection(*schemas))
    assert common_fields, "No common columns across files"

    # build UNION ALL query
    union_query = " UNION ALL ".join(
        [
            f'SELECT {", ".join(common_fields)} '
            f'FROM read_csv_auto("{f}", SAMPLE_SIZE=-1)'
            for f in files
        ]
    )

    # export consolidated file
    con.execute(f"""
        COPY (
            {union_query}
        )
        TO '{output}'
        WITH (HEADER, DELIMITER ',');
    """)

    # sanity check
    total_rows = sum(row_counts.values())

    print(f"[EXPORT] file={output}")
    print(f"[EXPORT] rows={total_rows} | cols={len(common_fields)}")


consolidate_csvs(files, schemas, con, OUTPUT_FILE, row_counts)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

[EXPORT] file=data\processed\stack.csv
[EXPORT] rows=13301203 | cols=39


## **Preprocessing**

In [6]:
# converting high missingness % to VARCHAR (preventing possible coalescing from read_csv_auto)
varchar_cols = {
    'LEGACY_SR_NUMBER': 'VARCHAR',
    'SANITATION_DIVISION_DAYS': 'VARCHAR',
    'PARENT_SR_NUMBER': 'VARCHAR',
    'ELECTRICAL_DISTRICT': 'VARCHAR',
    'CREATED_DEPARTMENT': 'VARCHAR',
    'CITY': 'VARCHAR',
    'STATE': 'VARCHAR',
    'ELECTRICITY_GRID': 'VARCHAR',
    'ZIP_CODE': 'VARCHAR'
}

df = con.execute(f"""
    SELECT *
    FROM read_csv_auto(
        '{OUTPUT_FILE}',
        sample_size=-1,
        timestampformat='%Y-%m-%d %H:%M:%S',
        types={varchar_cols})
""").df()


(df.isna().mean().sort_values(ascending=False).head(25) * 100).round(decimals=2)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

LEGACY_SR_NUMBER            99.76
SANITATION_DIVISION_DAYS    98.86
PARENT_SR_NUMBER            94.87
ELECTRICAL_DISTRICT         85.40
CREATED_DEPARTMENT          40.88
CITY                        32.22
STATE                       32.22
ELECTRICITY_GRID            17.37
ZIP_CODE                     7.02
CLOSED_DATE                  1.45
STREET_TYPE                  0.83
PRECINCT                     0.17
COMMUNITY_AREA               0.15
WARD                         0.14
POLICE_BEAT                  0.14
POLICE_SECTOR                0.14
POLICE_DISTRICT              0.14
STREET_DIRECTION             0.10
LONGITUDE                    0.10
LOCATION                     0.10
LATITUDE                     0.10
X_COORDINATE                 0.10
Y_COORDINATE                 0.10
STREET_NUMBER                0.10
STREET_NAME                  0.09
dtype: float64

In [7]:
con.register("df_view", df)

con.execute("""
    COPY df_view
    TO 'data/processed/data.parquet'
    (FORMAT PARQUET)
""")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))